In [1]:
# ML_in_Finance-Interaction
# Author: Matthew Dixon
# Version: 1.0 (08.09.2019)
# License: MIT
# Email: matthew.dixon@iit.edu
# Notes: tested on Mac OS X with Python 3.6.9 and the following packages:
# numpy=1.18.1, keras=2.3.1, tensorflow=2.0.0, statsmodels=0.10.1, scikit-learn=0.22.1
# Citation: Please cite the following reference if this notebook is used for research purposes:
# Dixon M.F., I. Halperin and P. Bilokon, Machine Learning in Finance: From Theory to Practice, Springer Graduate textbook Series, 2020. 

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor
import statsmodels.api as sm
import sklearn

# Overview
The purpose of this notebook is to illustrate a neural network interpretability method which is compatible with linear regression, including an interaction term. 

In linear regression, provided the independent variables are scaled, one can view the regression coefficients as a measure of importance of the variables and their interaction effect. Equivalently, the dependent variable can be differentiated w.r.t. the inputs to give the coefficient, with the interaction obtained from the cross-term in the Hessian. 

Similarly, the derivatives of the network w.r.t. the inputs are a non-linear generalization of interpretability in a linear regression model with interaction effects. Moreover, we should expect the neural network gradients to approximate the regression model coefficients when the data is generated by a linear regression model with interaction terms. 

Various simple experimental tests, corresponding to Section 4 of Chpt 5, are performed to illustrate the properties of network interpretability.

## Simple Data Generation Process (DGP)

Generate data from a regression model with an interaction term 

$Y=X_1+X_2 + X_1X_2+\epsilon~, ~~X_1, X_2 \sim N(0,1)~, ~~\epsilon \sim N(0,\sigma_n^2)$

In [3]:
M = 5000 # Number of samples
np.random.seed(7) # Set NumPy's random seed for reproducibility
X = np.zeros(shape=(M, 2))
sigma_n = 0.01
X[:int(M/2), 0] = np.random.randn(int(M/2))
X[:int(M/2), 1] = np.random.randn(int(M/2))
# Use antithetic sampling to reduce sample bias in the mean
X[int(M/2):, 0] = -X[:int(M/2), 0]
X[int(M/2):, 1] = -X[:int(M/2), 1]

eps = np.random.randn(M)
Y = X[:, 0] + X[:, 1] + X[:, 0]*X[:, 1] + sigma_n*eps.flatten()

## Use ordinary least squares to fit a linear model to the data

For a baseline, let us compare the neural network with OLS regression. 

We fit statsmodels' OLS model to the data

In [4]:
ols_results = sm.OLS(Y, sm.add_constant(X)).fit()

For each input, get the predicted $Y$ value according to the model

In [5]:
y_ols = ols_results.predict(sm.add_constant(X))

View characteristics of the resulting model. You should observe that the intercept is close to zero and the other coefficients are close to one.

In [6]:
ols_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.669
Model:                            OLS   Adj. R-squared:                  0.669
Method:                 Least Squares   F-statistic:                     5052.
Date:                Mon, 18 May 2020   Prob (F-statistic):               0.00
Time:                        16:09:50   Log-Likelihood:                -7103.5
No. Observations:                5000   AIC:                         1.421e+04
Df Residuals:                    4997   BIC:                         1.423e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0243      0.014      1.713      0.087      -0.004       0.052
x1             0.9999      0.014     70.236      0.000       0.972       1.028
x2             1.0000      0.014     70.164      0.000       0.972       1.028
==============================================================================
Omnibus:                      846.889   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16614.377
Skew:                           0.168   Prob(JB):                         0.00
Kurtosis:                      11.924   Cond. No.                         1.02
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Compare with a feedforward NN with no hidden layers

Recall the feedforward network with no hidden layers or activation function is a linear regression model

Create a build function for the linear perceptron, which transforms the inputs directly to a single output

In [7]:
def linear_NN0_model(l1_reg=0.0):    
    model = Sequential()
    model.add(Dense(1, input_dim=2, kernel_initializer='normal')) 
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

An early stopping callback to terminate training once the weights appear to have converged to an optimum

In [8]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)

Passing the build function for our model and training parameters to the `KerasRegressor` constructor to create a Scikit-learn-compatible regression model. This allows you to take advantage of the library's built-in tools and estimator methods, and to incorporate it into Scikit-learn pipelines. 

In [9]:
lm = KerasRegressor(build_fn=linear_NN0_model, epochs=40, batch_size=10, verbose=1, callbacks=[es])

Train the model

In [10]:
lm.fit(X, Y)

Epoch 1/40
5000/5000 [==============================] - 3s 528us/step - loss: 2.1870 - mae: 1.0363 - mse: 2.1870
Epoch 2/40
5000/5000 [==============================] - 1s 286us/step - loss: 1.4523 - mae: 0.7912 - mse: 1.4523
Epoch 3/40
5000/5000 [==============================] - 1s 281us/step - loss: 1.1474 - mae: 0.6760 - mse: 1.1474
Epoch 4/40
5000/5000 [==============================] - 1s 272us/step - loss: 1.0432 - mae: 0.6392 - mse: 1.0432
Epoch 5/40
5000/5000 [==============================] - 1s 273us/step - loss: 1.0133 - mae: 0.6288 - mse: 1.0133
Epoch 6/40
5000/5000 [==============================] - 1s 293us/step - loss: 1.0068 - mae: 0.6265 - mse: 1.0068
Epoch 7/40
5000/5000 [==============================] - 1s 270us/step - loss: 1.0052 - mae: 0.6260 - mse: 1.0052
Epoch 8/40
5000/5000 [==============================] - 1s 265us/step - loss: 1.0048 - mae: 0.6261 - mse: 1.0048
Epoch 9/40
5000/5000 [==============================] - 1s 259us/step - loss: 1.0047 - mae: 0.62

### Check that the weights are close to one
The weights should be close to unity. The bias term is the second entry and should be close to zero.

In [11]:
W = lm.model.layers[0].get_weights()[0]
b = lm.model.layers[0].get_weights()[1]
print("Weights: "+ str(W))
print("Bias: " + str(b))

Weights: [[0.99630827]
 [0.99237144]]
Bias: [0.02239072]


## Compare with a feedforward NN with one hidden layer (unactivated)

This time we create a neural network with a hidden layer with 10 units.

In [12]:
n = 10 # number of hidden units

In [13]:
def linear_NN1_model(l1_reg=0.0):    
    model = Sequential()
    # Note the first argument passed to the Dense layer constructor
    model.add(Dense(n, input_dim=2, kernel_initializer='normal')) 
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

In [14]:
lm = KerasRegressor(build_fn=linear_NN1_model, epochs=50, batch_size=10, verbose=1, callbacks=[es])

Train the model

In [15]:
lm.fit(X, Y)

Epoch 1/50
5000/5000 [==============================] - 2s 490us/step - loss: 1.6932 - mae: 0.8501 - mse: 1.6932
Epoch 2/50
5000/5000 [==============================] - 2s 320us/step - loss: 1.0064 - mae: 0.6269 - mse: 1.0064
Epoch 3/50
5000/5000 [==============================] - 1s 265us/step - loss: 1.0069 - mae: 0.6264 - mse: 1.0069
Epoch 4/50
5000/5000 [==============================] - 2s 328us/step - loss: 1.0101 - mae: 0.6284 - mse: 1.0101
Epoch 5/50
5000/5000 [==============================] - 1s 276us/step - loss: 1.0089 - mae: 0.6268 - mse: 1.0089
Epoch 6/50
5000/5000 [==============================] - 2s 306us/step - loss: 1.0082 - mae: 0.6265 - mse: 1.0082
Epoch 7/50
5000/5000 [==============================] - 2s 305us/step - loss: 1.0069 - mae: 0.6275 - mse: 1.0069
Epoch 8/50
5000/5000 [==============================] - 1s 291us/step - loss: 1.0089 - mae: 0.6271 - mse: 1.0089
Epoch 9/50
5000/5000 [==============================] - 1s 278us/step - loss: 1.0096 - mae: 0.62

Extract the trained weights from the model

In [16]:
W1 = lm.model.get_weights()[0]
b1 = lm.model.get_weights()[1]
W2 = lm.model.get_weights()[2]
b2 = lm.model.get_weights()[3]
print(W1, W2)

[[-0.38235897  0.30883455 -0.27476394  0.37077746  0.33718014 -0.30436334
  -0.2637127  -0.30291387 -0.29929903  0.28886756]
 [-0.26061237  0.2982374  -0.3480544   0.25751233  0.20222877 -0.3529237
  -0.32209945 -0.2524027  -0.3108291   0.31747958]] [[-0.28780663]
 [ 0.30990782]
 [-0.32593182]
 [ 0.30765072]
 [ 0.3182374 ]
 [-0.3202046 ]
 [-0.2919182 ]
 [-0.36224043]
 [-0.33498996]
 [ 0.34460506]]


### Check that the coefficients are close to one and the intercept is close to zero

In [17]:
beta_0 = np.dot(np.transpose(W2), b1) + b2
beta_1 = np.dot(np.transpose(W2), W1[0])
beta_2 = np.dot(np.transpose(W2), W1[1])

In [18]:
print(beta_0, beta_1, beta_2)

[0.03100312] [1.0006595] [0.9364493]


## Compare with a feedforward NN with one hidden layer ($tanh$ activated)

Finally, we create another model with a 10 unit hidden layer, this time with a $tanh$ activation function.

In [19]:
# number of hidden neurons
n = 10

In [20]:
def linear_NN1_model_act(l1_reg=0.0):    
    model = Sequential()
    # Note the activation parameter passed to the layer constructor
    model.add(Dense(n, input_dim=2, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(1, kernel_initializer='normal')) 
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

In [21]:
lm = KerasRegressor(build_fn=linear_NN1_model_act, epochs=100, batch_size=10, verbose=1, callbacks=[es])

Train the model

In [22]:
lm.fit(X, Y)

Epoch 1/100
5000/5000 [==============================] - 2s 482us/step - loss: 1.7010 - mae: 0.8555 - mse: 1.7010
Epoch 2/100
5000/5000 [==============================] - 1s 251us/step - loss: 0.9881 - mae: 0.6248 - mse: 0.9881
Epoch 3/100
5000/5000 [==============================] - 1s 257us/step - loss: 0.9503 - mae: 0.6143 - mse: 0.9503
Epoch 4/100
5000/5000 [==============================] - 1s 246us/step - loss: 0.9102 - mae: 0.6005 - mse: 0.9102
Epoch 5/100
5000/5000 [==============================] - 1s 252us/step - loss: 0.8652 - mae: 0.5863 - mse: 0.8652
Epoch 6/100
5000/5000 [==============================] - 1s 273us/step - loss: 0.8183 - mae: 0.5731 - mse: 0.8183
Epoch 7/100
5000/5000 [==============================] - 1s 280us/step - loss: 0.7711 - mae: 0.5542 - mse: 0.7711
Epoch 8/100
5000/5000 [==============================] - 1s 284us/step - loss: 0.7274 - mae: 0.5408 - mse: 0.7274
Epoch 9/100
5000/5000 [==============================] - 1s 271us/step - loss: 0.6861 - 

5000/5000 [==============================] - 2s 305us/step - loss: 0.0182 - mae: 0.0563 - mse: 0.0182
Epoch 72/100
5000/5000 [==============================] - 2s 308us/step - loss: 0.0180 - mae: 0.0556 - mse: 0.0180
Epoch 73/100
5000/5000 [==============================] - 1s 280us/step - loss: 0.0180 - mae: 0.0556 - mse: 0.0180
Epoch 74/100
5000/5000 [==============================] - 1s 280us/step - loss: 0.0180 - mae: 0.0559 - mse: 0.0180
Epoch 75/100
5000/5000 [==============================] - 1s 274us/step - loss: 0.0176 - mae: 0.0548 - mse: 0.0176
Epoch 76/100
5000/5000 [==============================] - 1s 286us/step - loss: 0.0176 - mae: 0.0546 - mse: 0.0176
Epoch 77/100
5000/5000 [==============================] - 1s 292us/step - loss: 0.0174 - mae: 0.0546 - mse: 0.0174
Epoch 78/100
5000/5000 [==============================] - 1s 274us/step - loss: 0.0173 - mae: 0.0537 - mse: 0.0173
Epoch 79/100
5000/5000 [==============================] - 1s 289us/step - loss: 0.0171 - mae:

### Compute the sensitivities

In [23]:
# Assumes that the activation function is tanh
def sensitivities(lm, X):
    
    W1 = lm.model.get_weights()[0]
    b1 = lm.model.get_weights()[1]
    W2 = lm.model.get_weights()[2]
    b2 = lm.model.get_weights()[3]
    
    M = np.shape(X)[0]
    p = np.shape(X)[1]

    beta = np.array([0]*M*(p+1), dtype='float32').reshape(M, p+1)
    beta_interact = np.array([0]*M*p*p, dtype='float32').reshape(M, p, p)
    
    beta[:, 0] = (np.dot(np.transpose(W2), np.tanh(b1)) + b2)[0] # intercept \beta_0= F_{W,b}(0)
    for i in range(M):
 
        Z1 = np.tanh(np.dot(np.transpose(W1), np.transpose(X[i,])) + b1)
        
        D = np.diag(1 - Z1**2) 
        D_prime = np.diag(-2 * Z1 * (1 - Z1**2))   # Needed for interaction term     
          
        for j in range(p):  
            beta[i, j+1] = np.dot(np.transpose(W2), np.dot(D, W1[j]))
            # Interaction term
            for k in range(p):
                beta_interact[i, j, k ] = np.dot(np.transpose(W2), np.dot(np.diag(W1[j]), np.dot(D_prime, W1[k])))  
    
    return beta, beta_interact

In [24]:
beta, beta_inter = sensitivities(lm, X)

### Check that the intercept is close to one and the coefficients are close to one

In [25]:
print(np.mean(beta, axis=0))

[-0.04780496  1.0240353   1.0067265 ]


In [26]:
print(np.std(beta, axis=0))

[1.7955754e-06 9.7666699e-01 9.8022658e-01]


In [27]:
print(np.mean(beta_inter, axis=0)) # off-diagonals are interaction terms

[[-0.01265506  0.9733205 ]
 [ 0.9733205  -0.02740895]]
